<a href="https://colab.research.google.com/github/Pablo2710/ProyectoDSI/blob/main/ProyectoDSParteI_Martinez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Detección Exploratoria de Fraude en Transacciones Financieras**

Este proyecto se inicia con un Análisis Exploratorio de Datos (EDA) sobre un conjunto sintético de $\mathbf{5 \text{ millones}}$ de transacciones financieras. Este dataset fue generado mediante un script de Python, para replicar comportamientos de fraude y anomalías del mundo real, proporcionando una base sólida para investigaciones de Data Science. El conjunto de datos abarca $\mathbf{18 \text{ atributos}}$ detallados, incluyendo variables críticas como los montos (amount), el tipo de transacción, características de comportamiento de la cuenta (como el velocity_score y el spending_deviation_score), metadatos contextuales (location, device_used) y la etiqueta binaria de fraude (is_fraud).
La etapa inicial tiene un doble propósito: primero, obtener una comprensión profunda de la distribución y composición de las transacciones fraudulentas; y segundo, identificar las señales de alerta más fuertes que guiarán la construcción de un modelo predictivo robusto. Para lograr esto, utilizaremos visualizaciones y resúmenes numéricos para validar hipótesis clave.
La exploración se centrará en responder si las transacciones marcadas como fraude presentan un patrón estadísticamente diferente al de las transacciones legítimas. Esto implica investigar la relación entre el amount, los indicadores de riesgo de comportamiento y la etiqueta is_fraud mediante análisis bivariados. Además, se desarrollarán gráficos multivariados que permitan diagnosticar la interacción de tres o más factores contextuales (como la combinación de $\text{"transaction_type"}$, $\text{"merchant_category"}$ y $\text{"location"}$) y su impacto en la tasa de fraude.
El proceso también incluirá un paso esencial de diagnóstico de la calidad de los datos, con una identificación explícita de los valores perdidos, particularmente en variables como $\text{"time_since_last_transaction"}$ y $\text{"fraud_type"}$. Los resultados de este EDA no solo proporcionarán una visión clara de los drivers del fraude, sino que también servirán de base empírica para la ingeniería de características y el enfoque de modelado en etapas posteriores del proyecto.

## *PREGUNTAS E HIPOTESIS*

***Problemas a Resolver***

1.  ¿Cuál es la tasa de fraude global y cómo se distribuyen las transacciones fraudulentas según la hora o el día?
2.   Hay diferencias significativas en el amount promedio o en el velocity_score entre las transacciones fraudulentas y las legítimas? (Análisis Bivariado)
1.   ¿Cuales combinaciones de factores (por ejemplo, transaction_type, location y device_used) muestran una mayor proporción de fraude?
2.  ¿Cómo influyen los indicadores de riesgo calculados (como spending_deviation_score y geo_anomaly_score) en la probabilidad de que una transacción sea catalogada como fraude? (Análisis Multivariado/Correlación)

***Hipótesis a Comprobar***


*   **H1 (Monto y Velocidad):** Las transacciones fraudulentas tendrán un "amount" mucho mayor que la mediana y un velocity_score significativamente más alto (más transacciones en poco tiempo), indicando actividad sospechosa.
*   **H2 (Categoría/Ubicación):** Habrá mayor probabilidad de fraude asociada con ciertas "merchant_category" cuando la "location" no es la habitual del usuario (indicada por un alto "geo_anomaly_score").
*   **H3 (Dispositivo y Tipo):** El fraude es más frecuente en transacciones de tipo "transfer" o "withdrawal" y cuando el "device_used" es "mobile" o "atm", en comparación con el uso de pos.









## *IMPORTACIÓN Y VISUALIZACIÓN DE DATOS*

In [1]:
# Importar librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

# Configuraciones de visualización
sns.set_theme(style="whitegrid") # Establece un tema estético para los gráficos
plt.rcParams['figure.figsize'] = (12, 6) # Establece un tamaño de figura por defecto
warnings.filterwarnings('ignore') # Ignora advertencias (útil para presentaciones)

In [2]:
# Importación de Dataset desde repositorio en GitHub
url= 'https://media.githubusercontent.com/media/Pablo2710/ProyectoDSI/refs/heads/main/financial_fraud_detection_dataset.csv'

try:
    # 1. Cargar el dataset directamente desde GitHub
    df = pd.read_csv(url)

    # 2. Confirmación rápida de la carga exitosa
    print(f"Dataset cargado exitosamente. Dimensiones: {df.shape}")
    print("\nPrimeras 5 filas:")
    display(df.head()) # Usamos display() para una mejor visualización en Colab/Jupyter

except Exception as e:
    print(f"Ocurrió un error al cargar el archivo: {e}")

Dataset cargado exitosamente. Dimensiones: (5000000, 18)

Primeras 5 filas:


,transaction_id,timestamp,sender_account,receiver_account,amount,transaction_type,merchant_category,location,device_used,is_fraud,fraud_type,time_since_last_transaction,spending_deviation_score,velocity_score,geo_anomaly_score,payment_channel,ip_address,device_hash
0,T100000,2023-08-22T09:22:43.516168,ACC877572,ACC388389,343.78,withdrawal,utilities,Tokyo,mobile,False,NaN,NaN,-0.21,3,0.22,card,13.101.214.112,D8536477
1,T100001,2023-08-04T01:58:02.606711,ACC895667,ACC944962,419.65,withdrawal,online,Toronto,atm,False,NaN,NaN,-0.14,7,0.96,ACH,172.52.47.194,D2622631
2,T100002,2023-05-12T11:39:33.742963,ACC733052,ACC377370,2773.86,deposit,other,London,pos,False,NaN,NaN,-1.78,20,0.89,card,185.98.35.23,D4823498
3,T100003,2023-10-10T06:04:43.195112,ACC996865,ACC344098,1666.22,deposit,online,Sydney,pos,False,NaN,NaN,-0.60,6,0.37,wire_transfer,107.136.36.87,D9961380
4,T100004,2023-09-24T08:09:02.700162,ACC584714,ACC497887,24.43,transfer,utilities,Toronto,mobile,False,NaN,NaN,0.79,13,0.27,ACH,108.161.108.255,D7637601
